In [1]:
import sae_lens
import torch
import jaxtyping
import random
import datasets
import plotly.colors as pc
import plotly.express as px
import seaborn as sns
import numpy as np
import pandas as pd
from typing import List, Tuple
from tqdm import tqdm

def obtain_data() -> (
    Tuple[List[sae_lens.SAE], torch.nn.Module, torch.utils.data.Dataset]
):
    """
    load sae, model and dataset
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    layers = 6
    saes = []
    release = "pythia-70m-deduped-res-sm"
    model_name = "pythia-70m-deduped"
    for layer in tqdm(range(layers)):
        sae_id = f"blocks.{layer}.hook_resid_post"
        saes.append(
            sae_lens.SAE.from_pretrained(release=release, sae_id=sae_id, device=device)[
                0
            ]
        )

    model = sae_lens.HookedSAETransformer.from_pretrained(model_name)
    ds = datasets.load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")["train"]

    return saes, model, ds

saes, model, ds = obtain_data()

/mnt/c/Users/huohu/Documents/code/SAEGeometry/config/saegeometry-1tp4usyN-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 6/6 [00:04<00:00,  1.24it/s]
The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


Loaded pretrained model pythia-70m-deduped into HookedTransformer


In [15]:
code_acts = torch.load("../res/acts/BAAI-TACO-pythia70m-res-all6-acts.pt")
math_acts = torch.load("../res/acts/math-pythia70m-res-all6-acts.pt")
wiki_acts = torch.load("../res/acts/wiki-pythia70m-res-all6-acts.pt")

top_num = 325
top_index_code = torch.topk(code_acts, top_num).indices

/tmp/ipykernel_36529/93926725.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  code_acts = torch.load("../res/acts/BAAI-TACO-pythia70m-res-all6-acts.pt")
/tmp/ipykernel_3

In [71]:
len(ds)

36718

In [84]:
import copy
nz_all = []
doc_len = 0
freq_mean_global = 0
layers = 6
device = "cuda" if torch.cuda.is_available() else "cpu"
freqs = torch.zeros(saes[0].cfg.d_sae).to(device)
abl_layer = 0
abl_times = 10
for layer in range(layers-1):
    nz_freqs = []
    abl_layer = layer
    for idx in tqdm(range(abl_times)):
        saes2 = copy.deepcopy(saes)
        with torch.no_grad():
            abl_num = 29
            list(map(lambda idy: saes2[abl_layer].W_dec[idy, :].zero_(), top_index_code[0][abl_num*idx:abl_num*(idx+1)]))
        ds_ratio = 1e-3
        length_ds = int(len(ds) * ds_ratio)
        for idx in range(length_ds):
            # loop begin, fuck indent
            example = ds[idx]
            tokens = model.to_tokens([example["text"]], prepend_bos=True)
            _, cache1 = model.run_with_cache_with_saes(tokens, saes=saes, use_error_term=False)
            model.reset_saes()
            _, cache2 = model.run_with_cache_with_saes(tokens, saes=saes2, use_error_term=False)
            local_doc_len = cache1["blocks.0.hook_resid_post.hook_sae_acts_post"].shape[1]
            freq = torch.zeros_like(freqs)
            
            prompt2 = f"blocks.{abl_layer + 1}.hook_resid_post.hook_sae_acts_post"
            freq = (((cache1[prompt2] > 1e-3) + 0 + cache2[prompt2] > 1e-3)==1)[0].sum(
                0
            ) / local_doc_len
            # freq[layer] = (cache[prompt2] > 1e-3)[0].sum(0) / local_doc_len
            new_doc_len = doc_len + local_doc_len
            if idx == 0:
                freq_mean_global = freq
            else:
                freq_mean_global = (
                    freq_mean_global * doc_len / new_doc_len
                    + freq * local_doc_len / new_doc_len
                )
            doc_len = new_doc_len
        nz_freqs.append(freq_mean_global)
    nz_all.append(nz_freqs)

100%|██████████| 10/10 [00:13<00:00,  1.35s/it]


In [87]:
res_ = []
for layer in range(layers-1):
    for idx in range(len(nz_freqs)):
        res = np.intersect1d(top_index_code[layer+1].cpu().numpy(), nz_all[layer][idx].nonzero().view(-1).cpu().numpy())
        print(f'common iou: {res.shape} abl_layer: {layer} influence freq: {nz_all[layer][idx].nonzero().view(-1).shape}')
# last_iou = res_[0]
# for idx in range(len(res_) - 1):
#     iou = np.intersect1d(last_iou, res_[idx])
#     last_iou = iou
#     print(f"iou with last abl: {iou.shape}")

common iou: (310,) abl_layer: 0 influence freq: torch.Size([8160])
common iou: (311,) abl_layer: 0 influence freq: torch.Size([8320])
common iou: (311,) abl_layer: 0 influence freq: torch.Size([8147])
common iou: (311,) abl_layer: 0 influence freq: torch.Size([8224])
common iou: (312,) abl_layer: 0 influence freq: torch.Size([8123])
common iou: (310,) abl_layer: 0 influence freq: torch.Size([8171])
common iou: (311,) abl_layer: 0 influence freq: torch.Size([8158])
common iou: (312,) abl_layer: 0 influence freq: torch.Size([8146])
common iou: (311,) abl_layer: 0 influence freq: torch.Size([8154])
common iou: (311,) abl_layer: 0 influence freq: torch.Size([8124])
common iou: (322,) abl_layer: 1 influence freq: torch.Size([7014])
common iou: (322,) abl_layer: 1 influence freq: torch.Size([7018])
common iou: (322,) abl_layer: 1 influence freq: torch.Size([7014])
common iou: (322,) abl_layer: 1 influence freq: torch.Size([7018])
common iou: (322,) abl_layer: 1 influence freq: torch.Size([70

In [65]:
last_iou = res_[0]
for idx in range(len(res_) - 1):
    iou = np.intersect1d(last_iou, res_[idx])
    last_iou = iou
    print(iou.shape)

(81,)
(81,)
(81,)
(81,)
(81,)
(81,)
(81,)
(81,)
(81,)


In [61]:
print(res_[1])

[  129   387   492   813  1130  2521  2987  3362  3641  3664  4078  4445
  5289  5325  5355  5516  5884  5958  6589  6900  7188  7262  7308  7483
  7700  8038  8235  8527  8821  9801 10122 10155 10235 10491 12169 12405
 12422 12930 13874 14165 14264 14485 15114 15376 15433 16401 17241 17278
 18295 18706 18920 19135 20697 20895 21162 21401 21962 22079 22170 23497
 23645 24241 24972 26296 26476 26619 26906 27266 27452 28531 29009 29251
 29987 30263 30939 31570 31698 31724 31883 31885 32636 32674]
